In [56]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df_1 = pd.read_csv('Data/ISTANBUL_GercekZamanliUretim-LODOS-KEMERBURGAZ-RES-40W0000000006156-01022018-01022021.csv')

In [57]:
df_1.head()

,Tarih,Saat,Toplam (MWh),Do�al Gaz,Barajl�,Linyit,Akarsu,�thal K�m�r,R�zgar,G�ne�,Fuel Oil,Jeotermal,Asfaltit K�m�r,Ta� K�m�r,Biyok�tle,Nafta,LNG,Uluslararas�
0,01.02.2018,00:00,1.17,"0,00","0,00","0,00","0,00","0,00","1,17","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,01.02.2018,01:00,1.82,"0,00","0,00","0,00","0,00","0,00","1,82","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,01.02.2018,02:00,3.03,"0,00","0,00","0,00","0,00","0,00","3,03","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,01.02.2018,03:00,5.00,"0,00","0,00","0,00","0,00","0,00","5,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,01.02.2018,04:00,5.00,"0,00","0,00","0,00","0,00","0,00","5,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [58]:
df = df_1[['Tarih' , 'Saat' , 'Toplam (MWh)']]
df.head()

,Tarih,Saat,Toplam (MWh)
0,01.02.2018,00:00,1.17
1,01.02.2018,01:00,1.82
2,01.02.2018,02:00,3.03
3,01.02.2018,03:00,5.00
4,01.02.2018,04:00,5.00


In [59]:
df['Date'] = pd.date_range(start = '20180201' , freq = 'H' , periods = len(df))


In [60]:
df.set_index('Date' , inplace = True)
df = df.asfreq('h')
len(df)

23912

In [61]:
df = df[['Toplam (MWh)']]
df['Toplam (MWh)'] = df['Toplam (MWh)'].astype('float64')

<img src = 'energy_calculations.png'>

### Our aim is to find the velocity given the predicted energy output from a wind powerplant.
#### Most power plants announce their expected power outputs 24 hrs before realization

In [62]:
#cp = constant = 0.4 
#rho = constant = 1.24
#swept area(Istanbul enercon: E82) = 5281 m^2
#number of turbines = 12
def windVelocity(p , numberOfTurbines,cp,rho,swept):
    den = cp*rho*swept
    num = 2*(p / numberOfTurbines)*10**6
    v = num/den
    v = v**(1./3)
    v = v
    return v

cp = 0.4
rho = 1.24
swept = 5281
numberOfTurbines =1
p = 5

In [47]:
windVelocity(p, numberOfTurbines,cp,rho, swept)


15.629102953398883

In [48]:
cp = 0.4
rho = 1.24
swept = 5281
#12 turbines in this plant
numberOfTurbines =12

In [51]:
results = []
for i , j  in df.iterrows():
    results.append(windVelocity(j['Toplam (MWh)'],numberOfTurbines,cp,rho, swept))
results = np.array(results)
    

In [52]:
results

array([4.20673921, 4.87424586, 5.77693984, ..., 0.        , 0.        ,
       0.        ])

In [63]:
df['Windspeed(m/s)'] = results
df.columns = ['Total(MWh)' , 'WindSpeed(m/s)']
df

,Total(MWh),WindSpeed(m/s)
Date,,
2018-02-01 00:00:00,1.17,4.206739
2018-02-01 01:00:00,1.82,4.874246
2018-02-01 02:00:00,3.03,5.776940
2018-02-01 03:00:00,5.00,6.826640
2018-02-01 04:00:00,5.00,6.826640
...,...,...
2020-10-24 03:00:00,0.00,0.000000
2020-10-24 04:00:00,0.00,0.000000
2020-10-24 05:00:00,0.00,0.000000
